In [1]:
import os

os.chdir('../..')

In [2]:
from os.path import join

import numpy as np
import pandas as pd

import modules.point_processing as pp

In [3]:
kinect_dir = join('data', 'kinect')

In [4]:
df_truth = pd.read_pickle('results/dataframes/df_truth.pkl')



In [5]:
trial_names = df_truth.index.get_level_values(0).unique().values

In [6]:
def combine_dataframes(load_dir, trial_names):
    
    dataframe_dict = {}
    
    for trial_name in trial_names:
        dataframe_dict[trial_name] = pd.read_pickle(join(load_dir, trial_name) + '.pkl')

    return pd.concat(dataframe_dict)


In [7]:
df_hypo = combine_dataframes(join(kinect_dir, 'processed', 'hypothesis'), trial_names)
df_selected = combine_dataframes(join(kinect_dir, 'best_pos'), trial_names)
df_assigned = combine_dataframes(join(kinect_dir, 'assigned'), trial_names)

In [8]:
foot_parts = ['L_FOOT', 'R_FOOT']

In [9]:
# Truth positions on frames with head and both feet
df_truth = df_truth.loc[:, ['HEAD'] + foot_parts].dropna()

In [10]:
index_intersect = df_truth.index.intersection(df_selected.index)

In [11]:
df_selected.shape

(672, 3)

In [12]:
# Drop index level for walking pass
# This gives it the same index as the other dataframes
df_assigned.index = df_assigned.index.droplevel(1)

In [13]:
df_truth = df_truth.loc[index_intersect]
df_selected = df_selected.loc[index_intersect]
df_assigned = df_assigned.loc[index_intersect]


In [14]:
frames = df_truth.index

In [15]:
df_truth_modified = pd.DataFrame(index=frames, 
                                 columns=foot_parts)

for frame in frames:
    
    foot_proposals = df_hypo.loc[frame, 'FOOT']
    true_foot_l, true_foot_r = df_truth.loc[frame, foot_parts]
        
    proposal_closest_l, _ = pp.closest_point(foot_proposals, true_foot_l.reshape(1, -1))
    proposal_closest_r, _ = pp.closest_point(foot_proposals, true_foot_r.reshape(1, -1))
    
    df_truth_modified.loc[frame, foot_parts[0]] = proposal_closest_l
    df_truth_modified.loc[frame, foot_parts[1]] = proposal_closest_r

In [16]:
pp.position_accuracy(np.stack(df_truth_modified.L_FOOT), np.stack(df_truth.L_FOOT))

0.8347676419965576

In [17]:
pp.position_accuracy(np.stack(df_truth_modified.R_FOOT), np.stack(df_truth.R_FOOT))

0.8261617900172117

In [18]:
frames_intersect = df_truth.index.intersection(df_selected.index)

In [19]:
df_selected_assigned = pd.DataFrame(index=frames,
                                    columns=foot_parts)

for frame in frames:
    
    points_selected = np.stack(df_selected.loc[frame][foot_parts])
    points_truth = np.stack(df_truth_modified.loc[frame])
    
    assigned_l, assigned_r = pp.correspond_points(points_truth, points_selected)
    
    df_selected_assigned.loc[frame, foot_parts[0]] = assigned_l
    df_selected_assigned.loc[frame, foot_parts[1]] = assigned_r

In [20]:
pp.position_accuracy(np.stack(df_selected_assigned.L_FOOT), np.stack(df_truth_modified.L_FOOT))

0.9913941480206541

In [21]:
pp.position_accuracy(np.stack(df_selected_assigned.R_FOOT), np.stack(df_truth_modified.R_FOOT))

0.9827882960413081

In [22]:
df_truth_2d = df_truth.applymap(lambda point: point[[2, 0]])
df_truth_modified_2d = df_truth_modified.applymap(lambda point: point[[2, 0]])

In [23]:
pp.position_accuracy(np.stack(df_assigned.L_FOOT), np.stack(df_truth_2d.L_FOOT))

0.8864027538726333

In [24]:
pp.position_accuracy(np.stack(df_assigned.L_FOOT), np.stack(df_truth_2d.L_FOOT))

0.8864027538726333

In [25]:
pp.position_accuracy(np.stack(df_assigned.L_FOOT), np.stack(df_truth_modified_2d.L_FOOT))

0.9948364888123924

In [26]:
pp.position_accuracy(np.stack(df_assigned.R_FOOT), np.stack(df_truth_modified_2d.R_FOOT))

0.9793459552495697